In [1]:
import time
from PIL import Image
import numpy as np

from tqdm import notebook

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

from scipy.optimize import fmin_l_bfgs_b

import os
import json

Using TensorFlow backend.
/home/parker/.conda/envs/donkey/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/parker/.conda/envs/donkey/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/parker/.conda/envs/donkey/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/parker/.conda/envs/donk

In [2]:
height = 224
width = 224

content_weight = 0.75
style_weight = 1.5
total_variation_weight = 1.0

In [3]:
def getLayers(content_image, style_image, combination_image):
    input_tensor = backend.concatenate([content_image,
                                style_image,
                                combination_image], axis=0)
    model = VGG16(input_tensor=input_tensor)

    layers = dict([(layer.name, layer.output) for layer in model.layers])

    return layers

In [4]:
def content_loss(content, combination):
    return backend.sum(backend.square(combination - content))

In [5]:
def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

In [6]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = height * width
    return backend.sum(backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [7]:
def total_variation_loss(x):
    a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))

In [8]:
def eval_loss_and_grads(x, f_outputs):
    x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):

    def __init__(self, f_outputs):
        self.loss_value = None
        self.grads_values = None
        self.f_outs = f_outputs

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x, self.f_outs)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [9]:
def loadImagesToArray(contentImage, styleImage):
    contentImage = contentImage.resize((height, width))
    styleImage = styleImage.resize((height, width))

    contentArray = np.asarray(contentImage, dtype="float32")
    contentArray = np.expand_dims(contentArray, axis = 0)

    styleArray = np.asarray(styleImage, dtype="float32")
    styleArray = np.expand_dims(styleArray, axis=0)

    contentArray[:, :, :, 0] -= 103.939
    contentArray[:, :, :, 1] -= 116.779
    contentArray[:, :, :, 2] -= 123.68
    contentArray = contentArray[:, :, :, ::-1]

    styleArray[:, :, :, 0] -= 103.939
    styleArray[:, :, :, 1] -= 116.779
    styleArray[:, :, :, 2] -= 123.68
    styleArray = styleArray[:, :, :, ::-1]
    return contentArray, styleArray

In [10]:
def performStyleTransfer(contentImage, styleImage):
    content_array, style_array = loadImagesToArray(contentImage, styleImage)
    content_image = backend.variable(content_array)
    style_image = backend.variable(style_array)
    combination_image = backend.placeholder((1, height, width, 3))

    layers = getLayers(content_image, style_image, combination_image)
    loss = backend.variable(0.)

    layer_features = layers['block2_conv2']
    content_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]

    loss = loss + content_weight * content_loss(content_image_features,combination_features)


    feature_layers = ['block1_conv2', 'block2_conv2',
                  'block3_conv3', 'block4_conv3',
                  'block5_conv3']
    for layer_name in feature_layers:
        layer_features = layers[layer_name]
        style_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_features, combination_features)
        loss += (style_weight / len(feature_layers)) * sl

    
    loss = loss + total_variation_weight * total_variation_loss(combination_image)

    grads = backend.gradients(loss, combination_image)

    outputs = [loss]
    outputs = outputs + grads
    f_outputs = backend.function([combination_image], outputs)

    evaluator = Evaluator(f_outputs)

    x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.

    iterations = 10

    for i in range(iterations):
        #print('Start of iteration', i)
        #start_time = time.time()
        x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                        fprime=evaluator.grads, maxfun=20)
        #print('Current loss value:', min_val)
        #end_time = time.time()
        #print('Iteration %d completed in %ds' % (i, end_time - start_time))

    x = x.reshape((height, width, 3))
    x = x[:, :, ::-1]
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = np.clip(x, 0, 255).astype('uint8')
    backend.clear_session() 
    result = Image.fromarray(x)

    return result


In [11]:
directories = ["data/tub_11_20-03-29", "data/flipped"]
newDirectory = "data/styleTransferred"
style = "seattlewf.jpg"

numPhotos = 0
numRecs = 0
for directory in directories:
    print(directory)
    for filename in notebook.tqdm(os.listdir(directory)):
        if filename.endswith(".jpg"): 
            im = Image.open(directory+"/"+filename)
            styledImage = Image.open(style)
            im = performStyleTransfer(im, styledImage)
            styledName = os.path.splitext(filename)[0]+"_styled.jpg"
            im.save(newDirectory+"/"+styledName)
            numPhotos += 1
        elif filename.endswith(".json"):
            with open(directory+"/"+filename) as json_file:
                data = json.load(json_file)
            styledName = os.path.splitext(filename)[0]+"_styled.json"
            with open(newDirectory+"/"+styledName, 'w') as outfile:
                json.dump(data, outfile)
            numRecs += 1

print(numPhotos)
print(numRecs)

data/tub_11_20-03-29


Instructions for updating:
Colocations handled automatically by placer.



NameError: name 'keras' is not defined